## 『本次練習內容』
#### 使用Xception backbone做 Trnasfer Learning


## 『本次練習目的』
  #### 了解如何使用Transfer Learning
  #### 了解Transfer Learning的優點，可以觀察模型收斂速度

##### 可以自行嘗試多種架構

In [3]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import keras
from keras.layers import Input
 
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


input_tensor = Input(shape=(32, 32, 3))
#include top 決定要不要加入 fully Connected Layer

'''Xception 架構'''

'''Resnet 50 架構'''
model=keras.applications.ResNet50(include_top=False, weights='imagenet',
                                    input_tensor=input_tensor,
                                    pooling=None, classes=10)
model.summary()


Using TensorFlow backend.
C:\Users\lu\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\lu\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\lu\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\lu\AppData\Roaming\Python\

94658560/94653016 [==============================] - 91s 1us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
_________________________

## 添加層數

In [10]:
x = model.output

'''可以參考Cifar10實作章節'''
x=GlobalAveragePooling2D()(x)
x=Dense(output_dim=128,activation='relu')(x)
x=Dropout(p=0.1)(x)

predictions = Dense(output_dim=10,activation='softmax')(x)
model = Model(inputs=model.input, outputs=predictions)
print('Model深度：', len(model.layers))


Model深度： 181


C:\Users\lu\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  """
C:\Users\lu\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
  
C:\Users\lu\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  


## 鎖定特定幾層不要更新權重

In [11]:
for layer in model.layers[:100]:
    layer.trainable = False
for layer in model.layers[100:]:
    layer.trainable = True

## 準備 Cifar 10 資料

In [12]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

(50000, 32, 32, 3)


C:\Users\lu\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## Training

In [13]:
# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train,y_train,batch_size=32,epochs=100)

W0319 13:13:24.393328 15368 deprecation_wrapper.py:119] From C:\Users\lu\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/100
50000/50000 [==============================] - 2161s 43ms/step - loss: 1.8000 - accuracy: 0.2820
Epoch 2/100
50000/50000 [==============================] - 2088s 42ms/step - loss: 1.4382 - accuracy: 0.4758
Epoch 3/100
50000/50000 [==============================] - 1996s 40ms/step - loss: 1.2587 - accuracy: 0.5930
Epoch 4/100
50000/50000 [==============================] - 1995s 40ms/step - loss: 1.0702 - accuracy: 0.6438
Epoch 5/100
50000/50000 [==============================] - 2056s 41ms/step - loss: 0.9549 - accuracy: 0.6817
Epoch 6/100
50000/50000 [==============================] - 2155s 43ms/step - loss: 0.8654 - accuracy: 0.7109
Epoch 7/100
50000/50000 [==============================] - 2176s 44ms/step - loss: 0.7740 - accuracy: 0.7440
Epoch 8/100
50000/50000 [==============================] - 2329s 47ms/step - loss: 0.7275 - accuracy: 0.7605
Epoch 9/100
50000/50000 [==============================] - 2123s 42ms/step - loss: 0.6547 - accuracy: 0.7867
Epoch 10/100
50000/

50000/50000 [==============================] - 1990s 40ms/step - loss: 0.0966 - accuracy: 0.9705
Epoch 76/100
50000/50000 [==============================] - 1994s 40ms/step - loss: 0.1009 - accuracy: 0.9699
Epoch 77/100
50000/50000 [==============================] - 1991s 40ms/step - loss: 0.1069 - accuracy: 0.9667
Epoch 78/100
50000/50000 [==============================] - 1988s 40ms/step - loss: 0.0993 - accuracy: 0.9689
Epoch 79/100
50000/50000 [==============================] - 1989s 40ms/step - loss: 0.0914 - accuracy: 0.9732
Epoch 80/100
50000/50000 [==============================] - 1988s 40ms/step - loss: 0.0873 - accuracy: 0.9753
Epoch 81/100
50000/50000 [==============================] - 1990s 40ms/step - loss: 0.1001 - accuracy: 0.9709
Epoch 82/100
50000/50000 [==============================] - 1991s 40ms/step - loss: 0.0860 - accuracy: 0.9746
Epoch 83/100
50000/50000 [==============================] - 1993s 40ms/step - loss: 0.0864 - accuracy: 0.9758
Epoch 84/100
50000/5000